In [1]:
# huggingface에서 model 불러올 때는 module이 하나 더 필요. 그냥 진행할 때 huggingface에서 설치하라고 안내하는 것들을 미리 하는 것 :) 
!pip install datasets transformers torch evaluate transformers[torch] accelerate


In [2]:
import numpy as np
print(np.__version__)

1.26.0


In [3]:

from datasets import load_dataset
import pandas as pd

train_dict = load_dataset("ilmin/KoBert_review_train_set_v2.03")
test_dict = load_dataset("ilmin/KoBert_review_test_set_v2.03")


# DatasetDict을 DataFrame으로 변환
train_df = {key: train_dict[key].to_pandas() for key in train_dict.keys()}
test_df = {key: test_dict[key].to_pandas() for key in test_dict.keys()}

# 결과 출력
for key, train_df in train_df.items():
    print(f"{key.capitalize()} DataFrame:")
    print(train_df)

for key, test_df in test_df.items():
    print(f"{key.capitalize()} DataFrame:")
    print(test_df)

Train DataFrame:
                                             Review_Text  Label
0      포장하려고 들어선 순간부터 뭔가 말투가 짜증나있고 싸우자는 태도였지만 일단 참았는데...      0
1      재오픈 하신건지 그동안 못가서 아쉬웠습니다 그리고 아 솔직히 여기 보쌈이랑 전 김치...      1
2      맛과 분위기 좋고 스테이크 가성비 좋습니다 다만 생각보다 많이 익혀서 나옵니다 평소...      1
3                                  친절하고 분위기 좋은데맛이없어요 비싸요      0
4        시 분에 왔는데 재료 소진으로 문 닫은 곳은 첨 보네 장사를 하고 싶을 때만 하시나요      0
...                                                  ...    ...
25985             그릇에 밀가루 자국 다 묻히고 주시네 위생 관련해서는 문제가 많아보임      0
25986  스콘이 보통 퍽퍽해서 별로 좋아하지 않는데 지인이 사다주어 먹어보고 맛있어서 오늘 ...      1
25987  식혀서 먹었는데도 쫄깃하고 문어가 크고 부드러움사장님이 친절하시고 성수동에 트럭이 ...      1
25988  안심 등심둘다 맛 있고 특히 돌판위 트러플소금과 먹으면 그 맛은 최고 친철하시고 너...      1
25989  빙수 진짜 맛있어요 커피도 맛있고 분위기도 좋은데 음 디저트는 팔지 마세요 티라미수...      1

[25990 rows x 2 columns]
Train DataFrame:
                                            Review_Text  Label
0     볶음밥 밖에 안먹었지만 볶음밥 자체는 야채도많고 잘나오는편홀서빙하시는분이 친절해서 좋았음      1
1                   이번주만 두번 시켜먹었어요맛가성비 최고입니다사장

In [4]:
data = pd.concat([train_df, test_df], axis=0, ignore_index=True)
data  ##### 최종파일! 

,Review_Text,Label
0,포장하려고 들어선 순간부터 뭔가 말투가 짜증나있고 싸우자는 태도였지만 일단 참았는데...,0
1,재오픈 하신건지 그동안 못가서 아쉬웠습니다 그리고 아 솔직히 여기 보쌈이랑 전 김치...,1
2,맛과 분위기 좋고 스테이크 가성비 좋습니다 다만 생각보다 많이 익혀서 나옵니다 평소...,1
3,친절하고 분위기 좋은데맛이없어요 비싸요,0
4,시 분에 왔는데 재료 소진으로 문 닫은 곳은 첨 보네 장사를 하고 싶을 때만 하시나요,0
...,...,...
34649,배달시켰더니 만원이하 카드낸다고 역정냄 뭐 이런 엿같은 경우가 있는지 사장님 종업원...,0
34650,히레카츠 특유의 부드러운 식감은얄팍한 두께로 느낄 수 없네요,0
34651,주문하면 바로 만든다는던 그 맛이아닌것 같다주인이 바뀌었는지찹살탕수육도 낭동보관했...,0
34652,아 난 정말 모르겠음,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34654 entries, 0 to 34653
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Review_Text  34654 non-null  object
 1   Label        34654 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 541.6+ KB


In [6]:
data.isna().sum()

Review_Text    0
Label          0
dtype: int64

In [7]:
# 데이터셋의 형태 변경 (huggingface 형식에 맞게)
from datasets import Dataset

dataset = Dataset.from_pandas(data)
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['Review_Text', 'Label'],
        num_rows: 31188
    })
    test: Dataset({
        features: ['Review_Text', 'Label'],
        num_rows: 3466
    })
})

In [8]:
# 데이터 준비 끝 => 모델 불러오자
!pip install tokenizers
from tokenizers import Tokenizer
from transformers import AutoTokenizer,ElectraModel, ElectraTokenizer # GPU 있다면 사용가능
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device("cpu")
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-discriminator", use_fast=True) # 하드웨어 가속화 = True

tokenizer # vectorDB에 저장할 수 있는 형태로 만들기 위해 tokenizer 사용

2024-11-26 14:35:09.616288: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-26 14:35:09.623226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732631709.636390   19944 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732631709.640409   19944 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-26 14:35:09.654171: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appro

ElectraTokenizer(name_or_path='monologg/koelectra-base-discriminator', vocab_size=32200, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [9]:
dataset['train'][11]['Review_Text']

'가격도 적당하고 생면파스타 너무맛있었어요'

In [10]:
print(tokenizer("Python is easy"))

tokenizer(dataset['train'][11]['Review_Text'])

{'input_ids': [2, 52, 5808, 13648, 10794, 18852, 73, 11978, 5808, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


{'input_ids': [2, 26597, 18736, 10546, 2830, 5799, 5786, 11419, 10922, 6503, 5851, 15351, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
dataset['train'] = dataset['train'].rename_column('Label', 'labels')
dataset['test'] = dataset['test'].rename_column('Label', 'labels')

In [12]:
def tokenize(batch) :
    temp = tokenizer(batch['Review_Text'], padding=True, truncation=True, max_length=500) # padding=True : 텍스트데이터를 동일한 길이로
    temp['labels'] = batch['labels']  
    
    return temp

In [13]:
dataset = dataset.map(tokenize, batched=True, batch_size=None)
dataset

Map:   0%|          | 0/31188 [00:00<?, ? examples/s]

Map:   0%|          | 0/3466 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review_Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 31188
    })
    test: Dataset({
        features: ['Review_Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3466
    })
})

In [14]:
dataset['train'][11]

{'Review_Text': '가격도 적당하고 생면파스타 너무맛있었어요',
 'labels': 1,
 'input_ids': [2,
  26597,
  18736,
  10546,
  2830,
  5799,
  5786,
  11419,
  10922,
  6503,
  5851,
  15351,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [15]:
from transformers import AutoModelForSequenceClassification

id2label = {0:'negative', 1:'positive'}

model = AutoModelForSequenceClassification.from_pretrained( # 모델을 객체로 받아주기 : 'model = '
    "monologg/koelectra-base-discriminator",
    num_labels = 2,
    id2label = id2label # 예측한 클래스ID를 사람이 이해할 수 있는 형태로 변환해서 알려줌
)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
import evaluate
import numpy as np

# Hugging Face evaluate 라이브러리에서 accuracy(정확도) 메트릭을 불러오기
accuracy = evaluate.load('accuracy')

# coumpute_metrics 함수는 모델의 예측값과 실제 레이블을 받아 정확도 계산
def compute_metrics(eval_pred) :
    # eval_pred는 모델의 예측값과 실제 레이블을 포함하는 튜플
    predictions, labels = eval_pred # predictions : 모델이 예측한 값, labels : 실제 정답 레이블

    # predictions는 모델이 출력한 확률값 배열이며, 가장 높은 확률을 가진 클래스를 선택
    # np.argmax는 각 샘플에 대해 가장 높은 확률값을 가진 클래스의 인덱스를 반환
    predictions = np.argmax(predictions, axis=1)

    # accuracy.compute는 예측값(predictions)과 실제값(labels)을 비교하여 정확도 계산
    # predictions : 모델이 예측한 클래스, references=labels : 실제 정답 클래스
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
pip install tf-keras

Note: you may need to restart the kernel to use updated packages.


In [18]:
print(dataset['train'].features)

{'Review_Text': Value(dtype='string', id=None), 'labels': Value(dtype='int64', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}


In [20]:
from transformers import TrainingArguments, Trainer

# 모델 학습을 위한 하이퍼파라미터와 설정 정의
args = TrainingArguments(
    output_dir = 'train_dir',                   # 학습 결과를 저장할 디렉터리
    overwrite_output_dir = True,                # 출력 디렉터리에 이미 있는 파일을 덮어쓸지 여부
    num_train_epochs = 5,                       # 학습할 에포크(epoch) 수
    learning_rate = 2e-5,                       # 학습률
    per_device_train_batch_size = 8,            # 각 디바이스(예 : GPU)당 학습 배치 크기
    per_device_eval_batch_size = 8,             # 각 디바이스당 평가 배치 크기
    evaluation_strategy = 'epoch',              # 평가 전략 (여기서는 매 에포크마다 평가)
    save_steps=5000,                            # 1 epoch마다 저장
    eval_steps=5000,                            # 평가 주기
    logging_steps=500,                          # 로그 주기
    resume_from_checkpoint=True                 # 체크포인트에서 재개
)

# Trainer 객체를 생성하여 학습 및 평가를 관리
trainer = Trainer(
    model = model.train(),                            # 학습할 모델
    args = args,                                # 학습 파라미터 설정
    train_dataset = dataset['train'],           # 학습에 사용할 데이터셋
    eval_dataset = dataset['test'],             # 평가에 사용할 데이터셋
    compute_metrics = compute_metrics,          # 평가 지표를 계산하는 함수
    tokenizer = tokenizer                       # 토크나이저 (텍스트를 토큰으로 변환하는 도구)
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_19944/2556688564.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
trainer.train() # 모델 학습시키기

  0%|          | 0/19495 [00:00<?, ?it/s]